# What are the demographic characteristics of neighborhoods where entitlements are?

In [1]:
import json
import warnings
warnings.filterwarnings("ignore")

import geopandas
import intake
import ipyleaflet
import IPython.display
import ipywidgets
import matplotlib.pyplot as plt
import numpy
import pandas

import laplan

cat = intake.open_catalog("../catalogs/*.yml")

In [2]:
prefix_list = laplan.pcts.VALID_PCTS_PREFIX
suffix_list = laplan.pcts.VALID_PCTS_SUFFIX

remove_prefix = ["ENV"]
remove_suffix = [
    "EIR",
    "IPRO",
    "CA",
    "CATEX",
    "CPIO",
    "CPU",
    "FH",
    "G",
    "HD",
    "HPOZ",
    "ICO",
    "K",
    "LCP",
    "NSO",
    "S",
    "SN",
    "SP",
    "ZAI",
    "CRA", 
    "RFA",
]

prefix_list = [x for x in prefix_list if x not in remove_prefix]
suffix_list = [x for x in suffix_list if x not in remove_suffix]

In [3]:
# Load PCTS and subset to the prefix / suffix list we want
pcts = cat.pcts2.read()
pcts = laplan.pcts.subset_pcts(
    pcts,
    prefix_list=prefix_list,
    suffix_list=suffix_list,
    get_dummies=True,
    verbose=False,
)
pcts = laplan.pcts.drop_child_cases(pcts, keep_child_entitlements=True)

In [4]:
# ACS data for income, race, commute, tenure
census = cat.census_analysis_table.read()

# Census tracts
tracts = cat.census_tracts.read()
tracts = (
    tracts[["GEOID10", "geometry"]]
    .rename(columns = {"GEOID10": "GEOID"})
)

In [5]:
# Crosswalk linking AIN to tract GEOID
parcel_to_tract = cat.crosswalk_parcels_tracts.read()
parcel_to_tract = parcel_to_tract[["AIN", "num_AIN", "GEOID"]]

In [6]:
# Merge entitlements with tract using crosswalk
pcts = pandas.merge(
    pcts,
    parcel_to_tract, 
    on="AIN",
    how="inner",
    validate="m:1",
)

In [7]:
#  Clean AIN data and get rid of outliers
case_counts = pcts.CASE_ID.value_counts()
big_cases = pcts[pcts.CASE_ID.isin(case_counts[case_counts > 20].index)]

pcts = pcts[~pcts.CASE_ID.isin(big_cases.CASE_ID)]

In [8]:
# Our first pass at analyzing entitlements is to count the number
# of cases for each census tract, to see which kinds of entitlements
# are being applied for in which types of census tract:
entitlement_counts = (pcts
    [["GEOID", "CASE_ID", "CASE_YR_NBR"] + suffix_list]
    .astype({c: "int64" for c in suffix_list})
    .groupby("CASE_ID").agg({
        **{s: "max" for s in suffix_list},
        "CASE_YR_NBR": "first",
        "GEOID": lambda x: x.value_counts().index[0],
    })
    .groupby(["GEOID", "CASE_YR_NBR"])
    .sum()
).reset_index(level=1).rename(columns={"CASE_YR_NBR": "year"})
entitlement_counts = entitlement_counts.assign(
    year=entitlement_counts.year.astype("int64")
)

In [9]:
# Merge the census data with the entitlements counts:
joined = pandas.merge(
    census,
    entitlement_counts,
    on="GEOID",
    how="left", 
    validate="1:m"
).dropna().sort_values(["GEOID", "year"]).astype(
    {c: "int64" for c in suffix_list}
)

In [10]:
m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.CartoDB.Positron)
m.center = [34.07996230865876, -118.31123326410754]
m.zoom = 10

label = ipywidgets.HTML(value=f"<i>Hover to select</i>")
m.add_control(ipyleaflet.WidgetControl(widget=label, position="topright"))

# Plot entitlement stats against median household income,
# population density, and geography:
def plot_entitlement(df, tracts, suffix, year="2017"):
    if year == "all":
        to_plot = df[(df[suffix] != 0) & (df.year >= 2010)]
        to_plot = to_plot.groupby("GEOID").agg({
            suffix: "sum",
            "medhhincome": "first",
            "density": "first",
            "pct_whitenonhisp": "first",
        }).reset_index()
    else:
        to_plot = df[(df[suffix] != 0) & (df.year == int(year))]
    # Merge in geometry
    final_df = tracts.merge(
        to_plot,
        on="GEOID", 
        how="left",
    ).fillna(
        {suffix: 0}
    ).to_crs(epsg=4326)
    choro_data = final_df[suffix].to_dict()
    choro_data = {str(x): y for x,y in choro_data.items()}
    geo_data = json.loads(final_df.to_json())
    choro_layer = ipyleaflet.Choropleth(
        style={'fillOpacity': 0.6, "weight": 0},
        geo_data=geo_data,
        choro_data=choro_data,
    )
    def on_hover(**kwargs):
        properties = kwargs.get("feature", {}).get("properties")
        id = kwargs.get("feature", {}).get("id")
        if not properties:
            return
        label.value=f"""
        <b>Tract GEOID: </b>{properties["GEOID"]} <br>
        <b>Number of {suffix} entitlements: </b> {properties[suffix]} <br>
        """
    choro_layer.on_hover(on_hover)
    for l in m.layers:
        if isinstance(l, ipyleaflet.Choropleth):
            m.substitute_layer(l, choro_layer)
            break
    else:
        m.add_layer(choro_layer)

In [11]:
years = [("All (from 2010 - 2019)", "all")] + [(str(i), str(i)) for i in range(2010, 2020)]
year_dropdown = ipywidgets.Dropdown(description="Year", options=years)
suffix_dropdown = ipywidgets.Dropdown(description="Suffix")

display(year_dropdown)
display(suffix_dropdown)
display(m)

change_guard = False

def on_suffix_selection(*args):
    global change_guard
    if change_guard:
        return
    suffix = suffix_dropdown.value
    year = year_dropdown.value
    plot_entitlement(joined, tracts, suffix, year)

def on_year_selection(*args):
    global change_guard
    if year_dropdown.value == "all":
        condition = (joined.year >= 2010)
    else:
        condition = (joined.year == int(year_dropdown.value))
    counts = joined.loc[condition, suffix_list].sum()
    # Sort by alphabetical or in descending value of counts?
    counts = counts.sort_index()
    old_val = suffix_dropdown.value 
    change_guard=True
    suffix_dropdown.options = [
        (f"{name} ({count:,} applications)", name) 
        for name,count in zip(counts.index, counts)
    ]
    if old_val in counts.index:
        suffix_dropdown.value = old_val
    else:
        suffix_dropdown.index = 0
    change_guard=False
    on_suffix_selection()

on_year_selection()
suffix_dropdown.observe(on_suffix_selection, names="value")
year_dropdown.observe(on_year_selection, names="value")

Dropdown(description='Year', options=(('All (from 2010 - 2019)', 'all'), ('2010', '2010'), ('2011', '2011'), (…

Dropdown(description='Suffix', options=(), value=None)

Map(center=[34.07996230865876, -118.31123326410754], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## What are the outliers?

In [12]:
case_dropdown = ipywidgets.Dropdown(
    description="Outlier cases",
    options=tuple(
        (f"{r[1]} ({r[2]} parcels)", r[0]) for r in
        big_cases.groupby("CASE_ID").agg(
            {"CASE_NBR": "first", "AIN": "count"}
        ).sort_values("AIN", ascending=False).itertuples()
    )
)

outlier_output = ipywidgets.Output()

display(case_dropdown)
display(outlier_output)


def plot_case(case_id):
    to_map = geopandas.GeoDataFrame(
        big_cases[big_cases.CASE_ID == case_id].groupby("GEOID").agg({
            "AIN": "count",
            "PROJ_DESC_TXT": "first"
        }).merge(
            tracts,
            how="right",
            left_index=True,
            right_on="GEOID"
        )
    )
    description = to_map.PROJ_DESC_TXT.dropna().iloc[0]
    with outlier_output:
        outlier_output.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12,12))
        to_map.fillna(0).plot(column="AIN",legend=True, ax=ax, cmap="plasma")
        plt.close()
        print(description)
        display(fig)

def on_case_selection(*args):
    plot_case(case_dropdown.value)

case_dropdown.observe(on_case_selection, names=["value"])
on_case_selection()

Dropdown(description='Outlier cases', options=(('CPC-2010-2278-GPA (10159 parcels)', 180067.0), ('CPC-2010-589…

Output()

In [13]:
big_cases.groupby("CASE_ID").agg({
    "CASE_NBR": "first",
    "PROJ_DESC_TXT": "first",
    "AIN": "count"
}).sort_values("AIN", ascending=False).rename(
    columns={"PROJ_DESC_TXT": "Description", "AIN": "Parcels"}
).style